### Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

model_name= 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

C:\Users\padma\anaconda3\envs\llama\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
print(v.shape)
v[0]

(768,)


0.07822262

### Download the raw data and Prepare the documents

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

filtered_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
print(len(filtered_documents))


375


#### Q2 prepare index for the raw data and create the embeddings

In [4]:
from tqdm.auto import tqdm  
import numpy as np  


def create_embeddings(filtered_documents):
    
    embeddings = []  # Initialize an empty list to store the embeddings
    
    # Iterate over the filtered documents 
    for doc in tqdm(filtered_documents):
        question = doc['question']  
        text = doc['text']  
        qa_text = f'{question} {text}'  # Concatenate the question and text
        
        # Encode the concatenated text using the embedding model
        encoded_array = embedding_model.encode(qa_text)
        doc['qa_text'] = encoded_array
        embeddings.append(encoded_array)  # Append the embedding to the list
        
    return embeddings
        
# create embedding for our document
embeddings = create_embeddings(filtered_documents)  

X = np.array(embeddings)  # Convert the list of embeddings to a NumPy array
X.shape  

  0%|          | 0/375 [00:00<?, ?it/s]

(375, 768)

#### Q3. Search

In [5]:
cosine_similarities = X.dot(v)
print('Index of max simiarity is :: ',np.argmax(cosine_similarities))
print('Highest score of similarity is: ',cosine_similarities[np.argmax(cosine_similarities)])

Index of max simiarity is ::  14
Highest score of similarity is:  0.65065753


### Vector search

In [6]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        # Initialize the search engine with a list of documents and their embeddings
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        # Calculate the dot product of the query vector with all embeddings
        scores = self.embeddings.dot(v_query)
        
        # Get the indices of the top num_results documents with the highest scores
        idx = np.argsort(-scores)[:num_results]
        
        # Return the corresponding top matching documents
        return [self.documents[i] for i in idx]

def VectorSearchEngine_helper(query):
    # Create a VectorSearchEngine instance with the documents and embeddings
    search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
    
    # Search for the top 5 results with the given query
    results = search_engine.search(query, num_results=5)
    
    # Return the search results
    return results
    
# Get the search results for the query vector v
result = VectorSearchEngine_helper(v)

# Print the IDs of the search results, separated by ********
for i in result:
    print(i['id'])
    print("********")

ee58a693
********
0a278fb2
********
6ba259b1
********
9f261648
********
e7ba6b8a
********


### Load the ground truth data

In [7]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
len(ground_truth)

1830

In [8]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        query_vector = embedding_model.encode(q['question'])
        results = search_function(query_vector)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }
       

#### Q4. Hit-rate for our vector search engine

In [9]:
dic = evaluate(ground_truth,VectorSearchEngine_helper)
print(dic)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}


### Q5 Indexing the document embeddings with Elasticsearch

In [10]:
from elasticsearch import Elasticsearch

def connect_to_es():
    for _ in range(10):  # Retry up to 10 times
        try:
            es = Elasticsearch("http://localhost:9200", basic_auth=('elastic', 'DkIedPPSCb'))
            if es.ping():
                return es
        except Exception as e:
            print(f"Connection failed, retrying... ({e})")
            time.sleep(10)
    raise Exception("Failed to connect to Elasticsearch after several retries")
    
es_client = connect_to_es()

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qa_text": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

C:\Users\padma\AppData\Local\Temp\ipykernel_96416\191422291.py:41: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [11]:
from tqdm.auto import tqdm

def create_embeddings_index(filtered_documents):
    for doc in tqdm(filtered_documents):
        question = doc['question']
        text = doc['text']
        qa_text = f'{question} {text}'
        
        doc['qa_text'] = embedding_model.encode(qa_text)

    for doc in tqdm(filtered_documents):
        es_client.index(index=index_name, document=doc)
        
create_embeddings_index(filtered_documents)

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

In [12]:
def elastic_search_knn(vector):
    knn = {
        "field": 'qa_text',
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

#### ID of the document with the highest score for Q1 vector

In [15]:
print(elastic_search_knn(v)[0])

{'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp', 'section': 'General course-related questions', 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'id': 'ee58a693'}


C:\Users\padma\AppData\Local\Temp\ipykernel_96416\718480251.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_results = es_client.search(


#### Q6. Compute Hit-rate for Elasticsearch

In [127]:
dic = evaluate(ground_truth,elastic_search_knn)
print(dic)

  0%|          | 0/1830 [00:00<?, ?it/s]

C:\Users\padma\AppData\Local\Temp\ipykernel_79688\718480251.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_results = es_client.search(


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}
